In [ ]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import joblib
import imodelsx.process_results
import sys
from neuro.features import qa_questions
import numpy as np
import neuro.config
sys.path.append('../experiments/')
results_dir = '/home/chansingh/mntv1/deep-fMRI/encoding/may15_single_question'
experiment_filename = '../experiments/04_fit_single_question.py'

# load the results in to a pandas dataframe
r = imodelsx.process_results.get_results_df(results_dir)
cols_varied = imodelsx.process_results.get_experiment_keys(
    r, experiment_filename)
print('experiment varied these params:', cols_varied)
r = imodelsx.process_results.fill_missing_args_with_default(
    r, experiment_filename)

# get the breakdown of data in these groups
r.groupby(cols_varied).size()
questions = qa_questions.get_questions(version='v3_boostexamples', full=True)


# quick overview
r['corrs_train_selected_mean'] = r['corrs_train_selected'].apply(np.nanmean)
r['corrs_test_selected_mean'] = r['corrs_test_selected'].apply(np.nanmean)
mets = [c for c in r.columns if 'corrs' in c and (
        'mean' in c or 'frac' in c)]
(
    r.groupby(cols_varied)[mets].mean()
    .style
    .background_gradient(
        cmap='viridis', axis=0)
    .format("{:.4f}")
)

In [67]:
dfs = []

for i, row in r.iterrows():
    df = pd.DataFrame(row['corrs_test_selected'],
                      columns=['corrs_test_selected'])
    df['subject'] = row.subject
    df['voxel_idx'] = df.index
    df['question'] = list(
        map(lambda x: questions[x % len(questions)], row.qs_selected))
    dfs.append(pd.DataFrame(df))
df = pd.concat(dfs)

# add roi info
rois_df = joblib.load(join(neuro.config.root_dir, 'qa',
                      'roi_cache', f'voxel_roi_df.pkl'))
rois_df_dict = {
    (row.subject, row.voxel_num): row.roi for i, row in rois_df.iterrows()
}
df['roi'] = df.apply(lambda x: rois_df_dict.get(
    (x.subject, x.voxel_idx), []), axis=1)

In [91]:
num_rows = 20

# take the first 20 entries of each group
counts_mat = df.groupby('subject')[
    'question'].value_counts().unstack().fillna(0).T
counts_mat['Avg'] = counts_mat.mean(axis=1).round().astype(int)
counts_mat.columns = pd.MultiIndex.from_product(
    [['Voxel counts'], counts_mat.columns])

corr_mat = df.groupby(['subject', 'question'])[
    'corrs_test_selected'].mean().unstack().T
corr_mat = corr_mat.loc[counts_mat.index]
corr_mat['Avg'] = corr_mat.mean(axis=1)
corr_mat.columns = pd.MultiIndex.from_product(
    [['Test correlation'], corr_mat.columns])

roi_mat = df.explode(['roi']).groupby(['question'])[
    'roi'].value_counts().unstack().fillna(0).astype(int)
topk_rois = roi_mat.head(num_rows).sum(
).sort_values(ascending=False).index[:10]
roi_mat = roi_mat.loc[counts_mat.index, topk_rois]
roi_mat.columns = pd.MultiIndex.from_product(
    [['ROI counts (across UTS01-UTS03)'], roi_mat.columns])


full_mat = pd.concat([counts_mat, corr_mat, roi_mat], axis=1)
full_mat = full_mat.sort_values(('Voxel counts', 'Avg'), ascending=False)
# full_mat = full_mat.sort_values(('Corr', 'Avg'), ascending=False)


# Add row with index 'All' to first position in full_mat
full_mat.loc['All'] = full_mat.sum().astype(int)
full_mat = full_mat.reindex(['All'] + list(full_mat.index[:-1]))
full_mat = full_mat.head(num_rows)
full_mat.loc[['All'] * 3, [('Test correlation', f'UTS0{k}') for k in [1, 2, 3]]] = df.groupby(
    'subject')['corrs_test_selected'].mean().sort_index().values
full_mat.loc['All', ('Test correlation', 'Avg')] = df.groupby(
    'subject')['corrs_test_selected'].mean().mean()

display(
    full_mat.style.background_gradient(
        cmap='viridis', axis=0,
        subset=(full_mat.index[1:], full_mat.columns)).format(precision=2)
)